<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению. Сессия № 2
</center>
Автор материала: Юрий Исаков и Юрий Кашницкий. Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Тема 4. Линейные модели классификации и регрессии
## <center>  Практика. Идентификация пользователя с помощью логистической регрессии

Тут мы воспроизведем парочку бенчмарков нашего соревнования и вдохновимся побить третий бенчмарк, а также остальных участников. Веб-формы для отправки ответов тут не будет, ориентир – [leaderboard](https://www.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/leaderboard) соревнования.

In [238]:
import pickle

import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, hstack
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm_notebook

%matplotlib inline
import seaborn as sns
from matplotlib import pyplot as plt

### 1. Загрузка и преобразование данных
Зарегистрируйтесь на [Kaggle](www.kaggle.com), если вы не сделали этого раньше, зайдите на [страницу](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) соревнования и скачайте данные. Первым делом загрузим обучающую и тестовую выборки и посмотрим на данные.

In [239]:
# загрузим обучающую и тестовую выборки
train_df = pd.read_csv("./../data/train_sessions.csv", index_col="session_id")
test_df = pd.read_csv("./../data/test_sessions.csv", index_col="session_id")

# приведем колонки time1, ..., time10 к временному формату
times = ["time%s" % i for i in range(1, 11)]
train_df[times] = train_df[times].apply(pd.to_datetime)
test_df[times] = test_df[times].apply(pd.to_datetime)

# отсортируем данные по времени
train_df = train_df.sort_values(by="time1")

# посмотрим на заголовок обучающей выборки
train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
21669,56,2013-01-12 08:05:57,55.0,2013-01-12 08:05:57,NaN,NaT,NaN,NaT,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
54843,56,2013-01-12 08:37:23,55.0,2013-01-12 08:37:23,56.0,2013-01-12 09:07:07,55.0,2013-01-12 09:07:09,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
77292,946,2013-01-12 08:50:13,946.0,2013-01-12 08:50:14,951.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:16,...,2013-01-12 08:50:16,948.0,2013-01-12 08:50:16,784.0,2013-01-12 08:50:16,949.0,2013-01-12 08:50:17,946.0,2013-01-12 08:50:17,0
114021,945,2013-01-12 08:50:17,948.0,2013-01-12 08:50:17,949.0,2013-01-12 08:50:18,948.0,2013-01-12 08:50:18,945.0,2013-01-12 08:50:18,...,2013-01-12 08:50:18,947.0,2013-01-12 08:50:19,945.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:20,0
146670,947,2013-01-12 08:50:20,950.0,2013-01-12 08:50:20,948.0,2013-01-12 08:50:20,947.0,2013-01-12 08:50:21,950.0,2013-01-12 08:50:21,...,2013-01-12 08:50:21,946.0,2013-01-12 08:50:21,951.0,2013-01-12 08:50:22,946.0,2013-01-12 08:50:22,947.0,2013-01-12 08:50:22,0


В обучающей выборке содержатся следующие признаки:
    - site1 – индекс первого посещенного сайта в сессии
    - time1 – время посещения первого сайта в сессии
    - ...
    - site10 – индекс 10-го посещенного сайта в сессии
    - time10 – время посещения 10-го сайта в сессии
    - target – целевая переменная, 1 для сессий Элис, 0 для сессий других пользователей
    
Сессии пользователей выделены таким образом, что они не могут быть длиннее получаса или 10 сайтов. То есть сессия считается оконченной либо когда пользователь посетил 10 сайтов подряд либо когда сессия заняла по времени более 30 минут.

В таблице встречаются пропущенные значения, это значит, что сессия состоит менее, чем из 10 сайтов. Заменим пропущенные значения нулями и приведем признаки к целому типу. Также загрузим словарь сайтов и посмотрим, как он выглядит:

In [240]:
# приведем колонки site1, ..., site10 к целочисленному формату и заменим пропуски нулями
sites = ["site%s" % i for i in range(1, 11)]
train_df[sites] = train_df[sites].fillna(0).astype("int")
test_df[sites] = test_df[sites].fillna(0).astype("int")

# загрузим словарик сайтов
with open(r"./../data/site_dic.pkl", "rb") as input_file:
    site_dict = pickle.load(input_file)

# датафрейм словарика сайтов
sites_dict_df = pd.DataFrame(
    list(site_dict.keys()), index=list(site_dict.values()), columns=["site"]
)
print(u"всего сайтов:", sites_dict_df.shape[0])
sites_dict_df.head()

всего сайтов: 48371


,site
25075,www.abmecatronique.com
13997,groups.live.com
42436,majeureliguefootball.wordpress.com
30911,cdt46.media.tourinsoft.eu
8104,www.hdwallpapers.eu


Выделим целевую переменную и объединим выборки, чтобы вместе привести их к разреженному формату.

In [241]:
# наша целевая переменная
y_train = train_df["target"]

# объединенная таблица исходных данных
full_df = pd.concat([train_df.drop("target", axis=1), test_df])

# индекс, по которому будем отделять обучающую выборку от тестовой
idx_split = train_df.shape[0]

Для самой первой модели будем использовать только посещенные сайты в сессии (но не будем обращать внимание на временные признаки). За таким выбором данных для модели стоит такая идея:  *у Элис есть свои излюбленные сайты, и чем чаще вы видим эти сайты в сессии, тем выше вероятность, что это сессия Элис и наоборот.*

Подготовим данные, из всей таблицы выберем только признаки `site1, site2, ... , site10`. Напомним, что пропущенные значения заменены нулем. Вот как выглядят первые строки таблицы:

In [242]:
# табличка с индексами посещенных сайтов в сессии
full_sites = full_df[sites]
full_sites.head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
21669,56,55,0,0,0,0,0,0,0,0
54843,56,55,56,55,0,0,0,0,0,0
77292,946,946,951,946,946,945,948,784,949,946
114021,945,948,949,948,945,946,947,945,946,946
146670,947,950,948,947,950,952,946,951,946,947


Сессии представляют собой последовательность индексов сайтов и данные в таком виде неудобны для линейных методов. В соответствии с нашей гипотезой (у Элис есть излюбленные сайты) надо преобразовать эту таблицу таким образом, чтобы каждому возможному сайту соответствовал свой отдельный признак (колонка), а его значение равнялось бы количеству посещений этого сайта в сессии. Это делается в две строчки:

In [243]:
from scipy.sparse import csr_matrix

In [244]:
csr_matrix?

In [245]:
# последовательность с индексами
sites_flatten = full_sites.values.flatten()

In [246]:
# искомая матрица
full_sites_sparse = csr_matrix(
    (
        [1] * sites_flatten.shape[0],
        sites_flatten,
        range(0, sites_flatten.shape[0] + 10, 10),
    )
)[:, 1:]
# убираем, потому что индексирование пошло с 1

In [247]:
X_train_sparse = full_sites_sparse[:idx_split]
X_test_sparse = full_sites_sparse[idx_split:]

Еще один плюс использования разреженных матриц в том, что для них имеются специальные реализации как матричных операций, так и алгоритмов машинного обучения, что подчас позволяет ощутимо ускорить операции за счет особенностей структуры данных. Это касается и логистической регрессии. Вот теперь у нас все готово для построения нашей первой модели.

### 2. Построение первой модели

Итак, у нас есть алгоритм и данные для него, построим нашу первую модель, воспользовавшись релизацией [логистической регрессии](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) из пакета `sklearn` с параметрами по умолчанию. Первые 90% данных будем использовать для обучения (обучающая выборка отсортирована по времени), а оставшиеся 10% для проверки качества (validation). 

**Напишите простую функцию, которая будет возвращать качество модели на отложенной выборке, и обучите наш первый классификатор**.

In [248]:
def get_auc_lr_valid(X, y, C=1.0, ratio=0.9, seed=42, max_iter=1000):
    """
    X, y – выборка
    ratio – в каком отношении поделить выборку
    C, seed – коэф-т регуляризации и random_state 
              логистической регрессии
    """

    # Ваш код здесь
    
    border = int(X.shape[0]*ratio)
    X_train, y_train = X[:border], y[:border]
    X_valid, y_valid = X[:border], y[:border]
    
    lr = LogisticRegression(C=C, max_iter=max_iter, random_state=seed)
    lr.fit(X_train, y_train)
    
    y_pred = lr.predict_proba(X_valid)[:, 1]
    
    return roc_auc_score(y_valid, y_pred)

**Посмотрите, какой получился ROC AUC на отложенной выборке.**

In [249]:
%%time
print(f'ROC-AUC score: {get_auc_lr_valid(X_train_sparse, y_train)}')

ROC-AUC score: 0.9788439873496724
Wall time: 5.84 s


Будем считать эту модель нашей первой отправной точкой (baseline). Для построения модели для прогноза на тестовой выборке **необходимо обучить модель заново уже на всей обучающей выборке** (пока наша модель обучалась лишь на части данных), что повысит ее обобщающую способность:

In [250]:
# функция для записи прогнозов в файл
def write_to_submission_file(
    predicted_labels, out_file, target="target", index_label="session_id"):
    predicted_df = pd.DataFrame(
        predicted_labels,
        index=np.arange(1, predicted_labels.shape[0] + 1),
        columns=[target],
    )
    predicted_df.to_csv(out_file, index_label=index_label)

**Обучите модель на всей выборке, сделайте прогноз для тестовой выборки и сделайте посылку в соревновании**.

In [251]:
def fit_predict_proba(X_test, X_train, y_train, C=1.0, seed=42, max_iter=1000):
    lr = LogisticRegression(C=C, max_iter=max_iter, random_state=seed)
    lr.fit(X_train, y_train)
    predictions = lr.predict_proba(X_test)
    return predictions

In [252]:
predictions = fit_predict_proba(X_test_sparse, X_train_sparse, y_train)[:,1]

In [253]:
write_to_submission_file(predictions, './alise_lr_baseline_1.csv')

Если вы выполните эти действия и загрузите ответ на [странице](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) соревнования, то воспроизведете первый бенчмарк "Logit".

**Score: 0.90744**

### 3. Улучшение модели, построение новых признаков

Создайте такой признак, который будет представлять собой число вида ГГГГММ от той даты, когда проходила сессия, например 201407 -- 2014 год и 7 месяц. Таким образом, мы будем учитывать помесячный [линейный тренд](http://people.duke.edu/~rnau/411trend.htm) за весь период предоставленных данных.

Добавьте новый признак, предварительно отмасштабировав его с помощью `StandardScaler`, и снова посчитайте ROC AUC на отложенной выборке.

In [263]:
def convert_date(x):
    date = list(map(int, str(x).split(' ')[0].split('-')))
    year, month = date[0], date[1]
    return year*100 + month

In [264]:
full_df = pd.concat([train_df.drop("target", axis=1), test_df])
idx_split = train_df.shape[0]

In [265]:
# Конвертируем данные раздельно, чтобы не было утечки информации
train_df_converted_date = train_df['time1'].apply(convert_date).values.reshape(-1, 1)
test_df_converted_date = test_df['time1'].apply(convert_date).values.reshape(-1, 1)

In [266]:
obj = StandardScaler()

train_df['YYYYMM'] = obj.fit_transform(train_df_converted_date)
test_df['YYYYMM'] = obj.fit_transform(test_df_converted_date)

In [267]:
full_df = pd.concat([train_df.drop("target", axis=1), test_df])
full_sites = full_df[sites + ['YYYYMM']]
sites_flatten = full_sites.values.flatten()
full_sites_sparse = csr_matrix(
    (
        [1] * sites_flatten.shape[0],
        sites_flatten,
        range(0, sites_flatten.shape[0] + 11, 11),
    )
)[:, 1:]
X_train_sparse = full_sites_sparse[:idx_split]
X_test_sparse = full_sites_sparse[idx_split:]

In [268]:
%%time
print(f'ROC-AUC score: {get_auc_lr_valid(X_train_sparse, y_train)}')

ROC-AUC score: 0.9788439873496724
Wall time: 5.79 s


**Вывод: score не изменился**

**Добавьте два новых признака: start_hour и morning.**

Признак `start_hour` – это час в который началась сессия (от 0 до 23), а бинарный признак `morning` равен 1, если сессия началась утром и 0, если сессия началась позже (будем считать, что утро это если `start_hour равен` 11 или меньше).

**Посчитйте ROC AUC на отложенной выборке для выборки с:**
- сайтами, `start_month` и `start_hour`
- сайтами, `start_month` и `morning`
- сайтами, `start_month`, `start_hour` и `morning`

In [269]:
def get_start_hour(x):
    date = list(map(int, str(x).split(' ')[1].split(':')))
    hour = int(date[0])
    return hour

def is_morning(x):
    if x < 11:
        return True
    else:
        return False
    
def get_start_month(x):
    date = list(map(int, str(x).split(' ')[0].split('-')))
    month = int(date[1])
    return month

In [270]:
full_df['start_hour'] = full_df['time1'].apply(get_start_hour)

In [271]:
full_df['morning'] = full_df['start_hour'].apply(is_morning)

In [272]:
full_df['start_month'] = full_df['time1'].apply(get_start_month)

In [273]:
full_df.drop([], axis=1)

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,site8,time8,site9,time9,site10,time10,YYYYMM,start_hour,morning,start_month
session_id,,,,,,,,,,,,,,,,,,,,,
21669,56,2013-01-12 08:05:57,55,2013-01-12 08:05:57,0,NaT,0,NaT,0,NaT,...,0,NaT,0,NaT,0,NaT,-1.744405,8,True,1
54843,56,2013-01-12 08:37:23,55,2013-01-12 08:37:23,56,2013-01-12 09:07:07,55,2013-01-12 09:07:09,0,NaT,...,0,NaT,0,NaT,0,NaT,-1.744405,8,True,1
77292,946,2013-01-12 08:50:13,946,2013-01-12 08:50:14,951,2013-01-12 08:50:15,946,2013-01-12 08:50:15,946,2013-01-12 08:50:16,...,784,2013-01-12 08:50:16,949,2013-01-12 08:50:17,946,2013-01-12 08:50:17,-1.744405,8,True,1
114021,945,2013-01-12 08:50:17,948,2013-01-12 08:50:17,949,2013-01-12 08:50:18,948,2013-01-12 08:50:18,945,2013-01-12 08:50:18,...,945,2013-01-12 08:50:19,946,2013-01-12 08:50:19,946,2013-01-12 08:50:20,-1.744405,8,True,1
146670,947,2013-01-12 08:50:20,950,2013-01-12 08:50:20,948,2013-01-12 08:50:20,947,2013-01-12 08:50:21,950,2013-01-12 08:50:21,...,951,2013-01-12 08:50:22,946,2013-01-12 08:50:22,947,2013-01-12 08:50:22,-1.744405,8,True,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82793,812,2014-10-02 18:20:09,1039,2014-10-02 18:20:09,676,2014-10-02 18:20:09,0,NaT,0,NaT,...,0,NaT,0,NaT,0,NaT,0.742059,18,False,10
82794,300,2014-05-26 14:16:40,302,2014-05-26 14:16:41,302,2014-05-26 14:16:44,300,2014-05-26 14:16:44,300,2014-05-26 14:17:19,...,1218,2014-05-26 14:17:19,1221,2014-05-26 14:17:19,1216,2014-05-26 14:17:19,-1.142416,14,False,5
82795,29,2014-05-02 11:21:56,33,2014-05-02 11:21:56,35,2014-05-02 11:21:56,22,2014-05-02 11:22:03,37,2014-05-02 11:22:03,...,21,2014-05-02 11:22:04,23,2014-05-02 11:22:04,6780,2014-05-02 11:22:04,-1.142416,11,False,5


In [347]:
def get_score(full_df, C=1.0, columns_to_add=[], columns_to_drop=[]):
    X_train_sparse, X_test_sparse = get_sparse(full_df, columns_to_add=columns_to_add, columns_to_drop=columns_to_drop)
    
    return get_auc_lr_valid(X_train_sparse, y_train, C=C)

def get_sparse(full_df, columns_to_add=[], columns_to_drop=[]):
    full_sites = full_df[sites + columns_to_add].drop(columns_to_drop, axis=1)
    sites_flatten = full_sites.values.flatten()
    full_sites_sparse = csr_matrix(
        (
            [1] * sites_flatten.shape[0],
            sites_flatten,
            range(0, sites_flatten.shape[0] + 10 + len(columns_to_add), 10 + len(columns_to_add)),
        )
    )[:, 1:]
    X_train_sparse = full_sites_sparse[:idx_split]
    X_test_sparse = full_sites_sparse[idx_split:]
    
    return X_train_sparse, X_test_sparse

In [348]:
print(f"сайты, start_month и start_hour: \
      {get_score(full_df, columns_to_add=['YYYYMM', 'start_month', 'start_hour'])}")

сайты, start_month и start_hour:       0.9886507438970251


In [349]:
print(f"сайты, start_month и morning: \
      {get_score(full_df, columns_to_add=['YYYYMM', 'start_month', 'morning'])}")

сайты, start_month и morning:       0.9867613260240045


In [287]:
print(f"сайты, start_month, start_hour и morning: \
      {get_score(full_df, columns_to_add=['YYYYMM', 'start_month', 'start_hour', 'morning'])}")

сайты, start_month, start_hour и morning:       0.9902567170103217


**Вывод: сайты, start_month, start_hour и morning показали наилучший результат**

### 4. Подбор коэффицициента регуляризации

Итак, мы ввели признаки, которые улучшают качество нашей модели по сравнению с первым бейслайном. Можем ли мы добиться большего значения метрики? После того, как мы сформировали обучающую и тестовую выборки, почти всегда имеет смысл подобрать оптимальные гиперпараметры -- характеристики модели, которые не изменяются во время обучения. Например, на 3 неделе вы проходили решающие деревья, глубина дерева это гиперпараметр, а признак, по которому происходит ветвление и его значение -- нет. В используемой нами логистической регрессии веса каждого признака изменяются и во время обучения находится их оптимальные значения, а коэффициент регуляризации остается постоянным. Это тот гиперпараметр, который мы сейчас будем оптимизировать.

Посчитайте качество на отложенной выборке с коэффициентом регуляризации, который по умолчанию `C=1`:

In [299]:
print(f'C=1.0, ROC-AUC score: {get_score(full_df, C=1.0)}')

C=1.0, ROC-AUC score: 0.9788439873496724


Постараемся побить этот результат за счет оптимизации коэффициента регуляризации. Возьмем набор возможных значений C и для каждого из них посчитаем значение метрики на отложенной выборке.

Найдите `C` из `np.logspace(-3, 1, 10)`, при котором ROC AUC на отложенной выборке максимален. 

In [343]:
def opt_param_search(func, data, parameters):
    max_score = 0
    best_parameters = {}
    for param_type in parameters:
        for param_val in parameters[param_type]:
            curr_score = func(data, param_val)
            if curr_score > max_score:
                max_score = curr_score
                best_parameters[param_type] = param_val
    
    return best_parameters, max_score

In [344]:
param = {'C': np.logspace(-3, 1, 10)}

best_params, best_score = opt_param_search(get_score, full_df, param)
best_params, best_score

({'C': 10.0}, 0.9825215792316843)

Наконец, обучите модель с найденным оптимальным значением коэффициента регуляризации и с построенными признаками `start_hour`, `start_month` и `morning`. Если вы все сделали правильно и загрузите это решение, то повторите второй бенчмарк соревнования.

In [359]:
X_train_sparse, X_test_sparse = get_sparse(full_df, columns_to_add=['YYYYMM', 'start_month', 'start_hour', 'morning'])
predictions = fit_predict_proba(X_test_sparse, X_train_sparse, y_train, C=best_params['C'])[:,1]
write_to_submission_file(predictions, './alise_lr_baseline_2.csv')

***Итоговый score: 0.8873, модель переобучилась***